In [1]:
import os
import csv
import gym
import glob
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

from stable_baselines import A2C
from stable_baselines import DQN

import tensorflow as tf

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
class IdsEnv(gym.Env):
    def __init__(self, dataset, images_per_episode=1, random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(dataset[0].columns) - 1,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

In [3]:
def mal_ben(df, label, normal):
    print("Preprocessing: make labels binary")
    print()
    df[label] = df[label].astype('object')
    atk_idx = df.loc[df[label] != normal].index
    df.loc[atk_idx, label] = 1.0
    df.loc[df.index.difference(atk_idx), label] = 0.0
    df[label] = df[label].astype(dtype=np.float32)

def preproc(df_train, df_test, columns):
    print("Preprocessing: One hot encoding + abundant features")
    print()
    df_train = pd.get_dummies(df_train,columns=columns)
    df_test = pd.get_dummies(df_test,columns=columns)
    extra_removables = df_test.columns ^ df_train.columns
    for to_remove in extra_removables:
            try:
                df_test.drop(to_remove, inplace=True, axis=1)
            except:
                print(f"{to_remove} already not part of test set, skipping")
            try:
                df_train.drop(to_remove, inplace=True, axis=1)
            except:
                print(f"{to_remove} already not part of train set, skipping")
    return df_train, df_test

def split_df(df):
    print("Preprocessing: split into X and Y sets")
    print()
    col = df.columns[-1]
    cols = df.columns[:-1]
    return df[cols], df[col]

def top_feat(df, dataset):
    print("Preprocessing: convert to top features")
    print()
    features = {
        'cicdos2017': ["Init Bwd Win Bytes",  "Idle Min", "ACK Flag Count", "Fwd Packet Length Min", "Fwd PSH Flags"],
        'cicids2017': ["Protocol", "Avg Bwd Segment Size", "Packet Length Max", "Bwd Packet Length Min", "Fwd IAT Mean"],
        'cicddos2019': ["URG Flag Count", "Down/Up Ratio", "Bwd Packet Length Min", "ACK Flag Count", "Fwd Packets Length Total"],
        'nslkdd': ["dst_host_serror_rate", "service_private", "count","dst_host_count", "service_domain_u", "flag_REJ", "dst_host_diff_srv_rate"],
        'unsw': ["sttl", "dttl", "ct_state_ttl", "service", "dload", "rate", "dmean", "dbytes", "dur", "is_sm_ips_ports", "dloss"],
    }
    important_features = features[dataset] + ["Label"] # Adding class for custom environment logic
    important_features = list(set(important_features))
    removable_features = df.columns ^ important_features
    
    return df.drop(labels=removable_features, axis='columns')

def ids_eval(model, x_test, y_test):
    print("Evaluation: starting validation of the model")
    print()
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        try:
            FPR = (float(FP) / (TN + FP)) # 1 - specificity
        except:
            FPR = 0.0
        f1_score = 2 * (precision * recall) / (precision + recall)
        print()
        print('Evaluation: validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))
    return [accuracy, precision, recall, FPR, f1_score]


In [ ]:
generalization_sets = ['cicddos2019', 'cicdos2017', 'cicids2017']
generalization_folders = ['cic-ddos2019', 'cic-dos2017', 'cic-ids2017']
datasets = ['cicddos2019-top', 'unswnb15', 'cicddos2019' , 'nslkdd' ] # , 'cicdos2017', 

end_d = ['model', 'train dataset', 'test dataset', 'features', 'accuracy', 'precision', 'recall', 'FPR', 'F1 score', 'notes']
end_df = pd.DataFrame(columns = end_d)

for d in datasets:
    dd = f'/project/masterproef-machine-learning-for-network-intrusion-detection/scripts/models/{d}/DQN'
    ad = f'/project/masterproef-machine-learning-for-network-intrusion-detection/scripts/models/{d}/A2C'
    os.chdir(dd)
    print()
    print("Evaluating DQN based models")
    print()
    for m in glob.glob("*.pkl"):
        print(f"Model {m}")
        print()
        print(f'Dataset {d}')
        print()
        model = DQN.load(f'{m}') 
        if d == 'nslkdd':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTrain.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTest.feather')
            mal_ben(df_train, 'class', 'normal')
            mal_ben(df_test, 'class', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['protocol_type' ,'service', 'flag'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'nslkdd', 'nslkdd', f'{len(df.columns)}'] + results + ['']
            print(entry)
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'unswnb15':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_training-set.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_testing-set.feather')
            df_train.drop('label', inplace=True, axis=1)
            df_test.drop('label', inplace=True, axis=1)
            mal_ben(df_train, 'attack_cat', 'normal')
            mal_ben(df_test, 'attack_cat', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['proto' ,'service', 'state'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'unswnb15', 'unswnb15', f'{len(df.columns)}'] + results + ['']
            print(entry)
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'cicddos2019':
            for i, x in enumerate(generalization_sets):
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_test, y_test = split_df(df)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print(entry)
                df_length = len(end_df)
                end_df.loc[df_length] = entry
        if d == 'cicddos2019-top':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicddos2019')
                mal_ben(df, 'Label', 'Benign')
                x_test, y_test = split_df(df)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print(entry)
                df_length = len(end_df)
                end_df.loc[df_length] = entry
        if d == 'cicdos2017':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicdos2017')
                mal_ben(df, 'Label', 'Benign')
                x_test, y_test = split_df(df)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicdos2017', x, f'{len(df.columns)}'] + results + ['']
                print(entry)
                df_length = len(end_df)
                end_df.loc[df_length] = entry  
    os.chdir(ad)
    for m in glob.glob("*.pkl"):
        model = A2C.load(f'{m}') 
        if d == 'nslkdd':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTrain.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTest.feather')
            mal_ben(df_train, 'class', 'normal')
            mal_ben(df_test, 'class', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['protocol_type' ,'service', 'flag'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'nslkdd', 'nslkdd', f'{len(df.columns)}'] + results + ['']
            print(entry)
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'unswnb15':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_training-set.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_testing-set.feather')
            df_train.drop('label', inplace=True, axis=1)
            df_test.drop('label', inplace=True, axis=1)
            mal_ben(df_train, 'attack_cat', 'normal')
            mal_ben(df_test, 'attack_cat', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['proto' ,'service', 'state'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'unswnb15', 'unswnb15', f'{len(df.columns)}'] + results + ['']
            print(entry)
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'cicddos2019':
            for i, x in enumerate(generalization_sets):
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_test, y_test = split_df(df)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print(entry)
                df_length = len(end_df)
                end_df.loc[df_length] = entry
        if d == 'cicddos2019-top':
            for i, x in enumerate(generalization_sets):
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicddos2019')
                mal_ben(df, 'Label', 'Benign')
                x_test, y_test = split_df(df)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print(entry)
                df_length = len(end_df)
                end_df.loc[df_length] = entry  
        if d == 'cicdos2017':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicdos2017')
                mal_ben(df, 'Label', 'Benign')
                x_test, y_test = split_df(df)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicdos2017', x, f'{len(df.columns)}'] + results + ['']
                print(entry)
                df_length = len(end_df)
                end_df.loc[df_length] = entry 


            
end_df.info()
end_df.describe()

end_df.to_csv('/project/masterproef-machine-learning-for-network-intrusion-detection/scripts/results.csv', index=False)

Loading a model without an environment, this model cannot be trained until it has a valid environment.






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


top
cicddos2019
6
(61,)
0.0    95092
1.0    42915
Name: Label, dtype: int64


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 72.49789861163445%
Precision: 97.23414100622581%
Recall/TPR/Sensitivity: 72.35604388724546%
FPR: 25.728820191743303%
F1 score: 0.8297034145465967
['dddqn.pkl', 'cicddos2019', 'cicddos2019', '6', 0.7249789861163445, 0.9723414100622582, 0.7235604388724547, 0.257288201917433, 0.8297034145465967, '']
cicdos2017
6
(61,)
0.0    208124
1.0     13745
Name: Label, dtype: int64

validation done...
Accuracy: 58.869768238486294%
Precision: 62.11751364649803%
Recall/TPR/Sensitivity: 91.23720798927235%
FPR: 98.33117142286969%
F1 score: 0.7391267059650888
['dddqn.pkl', 'cicddos2019', 'cicdos2017', '6', 0.588697682384863, 0.6211751364649803, 0.9123720798927235, 0.9833117142286969, 0.7391267059650888, '']
cicids2017
6
(61,)
0.0    1862623
1.0     336281
Name: Label, dtype: int64

validation done...
Accuracy: 77.33850803719311%
Precision: 85.10776056452963%
Recall/TPR/Sensitivity: 87.76893894933593%
FPR: 70.63122246836248%
F1 score: 0.8641786732518902
['dddqn.pkl', 'cicddos

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 85.27289064085099%
Precision: 78.6997307757025%
Recall/TPR/Sensitivity: 99.90654704688669%
FPR: 32.09825673534073%
F1 score: 0.8804414325380017
['dueling_dqn.pkl', 'cicddos2019', 'cicddos2019', '6', 0.8527289064085098, 0.786997307757025, 0.9990654704688668, 0.3209825673534073, 0.8804414325380017, '']
cicdos2017
6
(61,)
0.0    208124
1.0     13745
Name: Label, dtype: int64

validation done...
Accuracy: 25.177359306969073%
Precision: 23.073152917659723%
Recall/TPR/Sensitivity: 89.03764138698313%
FPR: 95.33036394817073%
F1 score: 0.36649086787614205
['dueling_dqn.pkl', 'cicddos2019', 'cicdos2017', '6', 0.2517735930696907, 0.23073152917659723, 0.8903764138698312, 0.9533036394817073, 0.36649086787614205, '']
cicids2017
6
(61,)
0.0    1862623
1.0     336281
Name: Label, dtype: int64

validation done...
Accuracy: 62.58996535543648%
Precision: 56.335603935902%
Recall/TPR/Sensitivity: 99.12188394434315%
FPR: 71.32631512481233%
F1 score: 0.7184074915156996
['dueling

...


validation done...
Accuracy: 99.94709131265162%
Precision: 100.0%
Recall/TPR/Sensitivity: 99.94709131265162%
FPR: 0.0%
F1 score: 0.9997353865615097
['pn_per_dddqn.pkl', 'unswnb15', 'unswnb15', '189', 0.9994709131265163, 1.0, 0.9994709131265163, 0.0, 0.9997353865615097, '']
Loading a model without an environment, this model cannot be trained until it has a valid environment.
0.0    34200
1.0    19746
Name: attack_cat, dtype: int64
0.0    51815
1.0    49225
Name: attack_cat, dtype: int64
proto_icmp already not part of train set, skipping
proto_rtp already not part of train set, skipping
state_ACC already not part of test set, skipping
state_CLO already not part of test set, skipping
state_ECO already not part of train set, skipping
state_PAR already not part of train set, skipping
state_URN already not part of train set, skipping
state_no already not part of train set, skipping
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53946 entries, 0 to 53945
Columns: 189 entries, dur to state

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 99.94321994528468%
Precision: 99.9961265832591%
Recall/TPR/Sensitivity: 99.94708926428267%
FPR: 100.0%
F1 score: 0.9997160191041693
['a2c.pkl', 'unswnb15', 'unswnb15', '189', 0.9994321994528468, 0.9999612658325909, 0.9994708926428267, 1.0, 0.9997160191041693, '']
Loading a model without an environment, this model cannot be trained until it has a valid environment.
0.0    95092
1.0    42915
Name: Label, dtype: int64


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 81.37155444769716%
Precision: 73.02709069493521%
Recall/TPR/Sensitivity: 99.9136690647482%
FPR: 37.44014948032232%
F1 score: 0.8438039225216905
['dddqn.pkl', 'cicddos2019', 'cicddos2019', '67', 0.8137155444769717, 0.7302709069493521, 0.999136690647482, 0.37440149480322316, 0.8438039225216905, '']
0.0    208124
1.0     13745
Name: Label, dtype: int64

validation done...
Accuracy: 76.84638475476189%
Precision: 76.95663873298993%
Recall/TPR/Sensitivity: 97.91282019930306%
FPR: 82.26293399204062%
F1 score: 0.8617904552816655
['dddqn.pkl', 'cicddos2019', 'cicdos2017', '67', 0.7684638475476189, 0.7695663873298992, 0.9791282019930305, 0.8226293399204062, 0.8617904552816655, '']
0.0    1862623
1.0     336281
Name: Label, dtype: int64

validation done...
Accuracy: 56.100180908471586%
Precision: 51.4133641997844%
Recall/TPR/Sensitivity: 94.07485122292886%
FPR: 76.63395814845174%
F1 score: 0.6648929707347115
['dddqn.pkl', 'cicddos2019', 'cicids2017', '67', 0.56100180